In [2]:
import glob
from datetime import datetime
import os

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from scipy import stats, signal
from scipy.stats import mode
from scipy.fft import fft
from tsmoothie.smoother import LowessSmoother

from sklearn.ensemble import RandomForestClassifier as RFC, ExtraTreesClassifier as ETC
from sklearn.model_selection import cross_val_score, StratifiedKFold, train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.preprocessing import MinMaxScaler

from bayes_opt import BayesianOptimization
from skopt  import BayesSearchCV 


import lightgbm as lgb
import xgboost as xgb

from tqdm import tqdm, trange

# Generic functions (usable for all test cases)

## Core functions (Change path here)

In [3]:
def segmentation(df, overlap_rate, time_window):
    seg_data = []
    #convert overlap rate to step for sliding window
    overlap = int((1 - overlap_rate)*time_window)
    # interpolate
    df = df.interpolate().ffill().fillna(0)
    for i in range(0, len(df)-time_window+1, overlap):
        seg_data.append(df.loc[i:i+time_window-1, :].copy().reset_index(drop=True))
    return seg_data


def rename_columns(df):
    df.columns = [
        "FH_X", "FH_Y", "FH_Z",     #1
        "TH_X", "TH_Y", "TH_Z",     #2
        "RH_X", "RH_Y", "RH_Z",     #3
        "RS_X", "RS_Y", "RS_Z",     #4
        "RO_X", "RO_Y", "RO_Z",     #5
        "RE_X", "RE_Y", "RE_Z",     #6
        "RW_X", "RW_Y", "RW_Z",     #7
        "LS_X", "LS_Y", "LS_Z",     #8
        "LE_X", "LE_Y", "LE_Z",     #9
        "LW_X", "LW_Y", "LW_Z",     #10
        "RA_X", "RA_Y", "RA_Z",     #11
        "LA_X", "LA_Y", "LA_Z",     #12
        "VS_X", "VS_Y", "VS_Z",     #13
        "subject_id", "activity",   # Other columns
    ]
    return df


def dataloader(overlap, window_size, verbose=True):
    if verbose:
        print("loading the data...", end="\t")
    data_list = []
    file_lengths = {1: [], 2: [], 3: []}
    ##### CHANGE PATH ######
    files = tqdm(glob.glob("../TrainData/*/*/*.csv")) if verbose else glob.glob("../TrainData/*/*/*.csv")
    ##### ##### ##### ######
    for file in files:
        tempdf = pd.read_csv(file)
        tempdf = rename_columns(tempdf)
        segmented_data = segmentation(tempdf, overlap, window_size)
        if len(segmented_data)>0:
            person = segmented_data[0].reset_index(drop=True).loc[0, "subject_id"]
            file_lengths[person].append(len(segmented_data))   
        data_list.extend(segmented_data)
    return data_list, file_lengths


def feature_extractor(data_list, verbose=True):
    if verbose:
        print(f"extracting the features...", end="  ")
    X, y = {1:[], 2:[], 3:[]}, {1:[], 2:[], 3:[]}
    num_range = trange(0,len(data_list)) if verbose else range(0,len(data_list))
    for j in num_range:
        #extract only xyz columns
        person = data_list[j].loc[0, "subject_id"]
        x_data = data_list[j].drop(columns=["subject_id","activity"])
        X[person].append(get_features(x_data))
        y[person].append(data_list[j].reset_index(drop=True).loc[0, "activity"])
    return X, y


def majority_voting(predictions, file_lengths):
    filtered_predictions = []
    index = 0
    for length in file_lengths:
        file_pred = predictions[index:index+length]
        modes = mode(file_pred)
        majority_choice = modes.mode[0]
        filtered_predictions.extend([majority_choice]*length)
        index += length
    return filtered_predictions

In [4]:
def get_processed_dataset(overlap_rate, window_size, verbose=True):
    data_list, file_lengths = dataloader(overlap_rate, window_size, verbose=verbose)
    stream_list = []
    for df in data_list:
        stream_list.append(get_streams(df))
    X, y = feature_extractor(stream_list, verbose=verbose)
    return X, y, file_lengths


def model_evaluator(model, X, y, file_lengths, n_repeats=10, voting=True, verbose=True):
    scores = []
    num_range = trange(n_repeats) if verbose else range(n_repeats)
    for _ in num_range:
        for p1, p2, p3 in [(1,2,3), (2,3,1), (3,1,2)]:
            X_test, y_test = X[p1], y[p1]
            X_train = X[p2] + X[p3]
            y_train = y[p2] + y[p3]
            # print(f"training model for person {p1}/3...", end="\t")
            model.fit(X_train, y_train)
            pred = model.predict(X_test)
            if voting:
                filtered_pred = majority_voting(pred, file_lengths[p1])
                scores.append(accuracy_score(y_test, filtered_pred))
            else:
                scores.append(accuracy_score(y_test, pred))
    if verbose:
        print(f"Mean Score: {np.mean(scores)}")
        print(f"Std Score: {np.std(scores)}")
        print(f"Min Score: {np.min(scores)}")
        print(f"Max Score: {np.max(scores)}")
    return scores


def scaled_model_evaluator(model, X, y, file_lengths, n_repeats=10, voting=True, verbose=True):
    scores = []
    num_range = trange(n_repeats) if verbose else range(n_repeats)
    for _ in num_range:
        for p1, p2, p3 in [(1,2,3), (2,3,1), (3,1,2)]:
            X_test, y_test = X[p1], y[p1]
            X_train = X[p2] + X[p3]
            y_train = y[p2] + y[p3]
            scaler = MinMaxScaler()
            X_train_scaled = scaler.fit_transform(X_train)
            X_test_scaled = scaler.fit_transform(X_test)
            # print(f"training model for person {p1}/3...", end="\t")
            model.fit(X_train_scaled, y_train)
            pred = model.predict(X_test_scaled)
            if voting:
                filtered_pred = majority_voting(pred, file_lengths[p1])
                scores.append(accuracy_score(y_test, filtered_pred))
            else:
                scores.append(accuracy_score(y_test, pred))
    if verbose:
        print(f"Mean Score: {np.mean(scores)}")
        print(f"Std Score: {np.std(scores)}")
        print(f"Min Score: {np.min(scores)}")
        print(f"Max Score: {np.max(scores)}")
    return scores

In [23]:
def bayesian_optimization(function, parameters):
   n_iterations = 5
   gp_params = {"alpha": 1e-4}

   BO = BayesianOptimization(function, parameters)
   BO.maximize(n_iter=n_iterations, **gp_params)

   return BO.max

## Stream Extractors

In [6]:
def get_speed_acc(x_data):
    x_data = x_data.drop(columns=["activity", "subject_id"])
    speed = x_data.diff().fillna(0)
    acc = speed.diff().fillna(0)
    speed.columns = [f"{col}_speed" for col in speed.columns]
    acc.columns = [f"{col}_acc" for col in acc.columns]
    return speed, acc

def get_speed_acc_jerk(x_data):
    x_data = x_data.drop(columns=["activity", "subject_id"])
    speed = x_data.diff().fillna(0)
    acc = speed.diff().fillna(0)
    jerk = acc.diff().fillna(0)
    speed.columns = [f"{col}_speed" for col in speed.columns]
    acc.columns = [f"{col}_acc" for col in acc.columns]
    jerk.columns = [f"{col}_jerk" for col in acc.columns]
    return speed, acc, jerk

In [7]:
def joint_distance(x_data, joint1, joint2):
    """
    returns the distance between two joints. 
    """
    x1, y1, z1 = x_data[f"{joint1}_X"], x_data[f"{joint1}_Y"], x_data[f"{joint1}_Z"]
    x2, y2, z2 = x_data[f"{joint2}_X"], x_data[f"{joint2}_Y"], x_data[f"{joint2}_Z"]
    distance = np.sqrt((x2-x1)**2 + (y2-y1)**2 + (z2-z1)**2)
    return distance

def get_all_joint_distances(x_data):
    """
    calculates all the necessary joint distances from the `x_data`, 
    adds columns to it and returns the modified `x_data`.
    the two joints should not be essentially consecutive, 
    because the distance between two consecutive joints is always constant.
    For example, distance between left_wrist and left_elbow is always constant.
    """
    # joints
    # Front head        ->  left shoulder       (1->8)
    x_data["dist_FH_LS"] = joint_distance(x_data, "FH", "LS")
    # Front head        ->  right shoulder      (1->4)
    x_data["dist_FH_RS"] = joint_distance(x_data, "FH", "RS")
    # left shoulder     ->  left wrist          (8->10)
    x_data["dist_LS_LW"] = joint_distance(x_data, "LS", "LW")
    # right shoulder    ->  right wrist         (4->7)
    x_data["dist_RS_RW"] = joint_distance(x_data, "RS", "RW")
    # v sacral          ->  left elbow          (13->9)
    x_data["dist_VS_LE"] = joint_distance(x_data, "VS", "LE")
    # v sacral          ->  right elbow         (13->6)
    x_data["dist_VS_RE"] = joint_distance(x_data, "VS", "RE")
    # v sacral          ->  left wrist          (13->10)
    x_data["dist_VS_LW"] = joint_distance(x_data, "VS", "LW")
    # v sacral          ->  right wrist         (13->7)
    x_data["dist_VS_RW"] = joint_distance(x_data, "VS", "RW")
    # v sacral          ->  rear head           (13->3)
    x_data["dist_VS_RH"] = joint_distance(x_data, "VS", "RH")
    # v sacral          ->  top head            (13->2)
    x_data["dist_VS_TH"] = joint_distance(x_data, "VS", "TH")
    # left wrist        ->  right wrist         (10->7)
    x_data["dist_LW_RW"] = joint_distance(x_data, "LW", "RW")
    # left asis         ->  left wrist          (12->10)
    x_data["dist_LA_LW"] = joint_distance(x_data, "LA", "LW")
    # right asis        ->  right wrist         (11->7)
    x_data["dist_RA_RW"] = joint_distance(x_data, "RA", "RW")
    # left wrist        ->  top head            (10->2)
    x_data["dist_LW_TH"] = joint_distance(x_data, "LW", "TH")
    # right wrist       ->  top head            (7->2)
    x_data["dist_RW_TH"] = joint_distance(x_data, "RW", "TH")
    # top head          ->  left asis           (2->12)
    x_data["dist_TH_LA"] = joint_distance(x_data, "TH", "LA")
    return x_data



In [8]:
def get_all_joint_distances_red(x_data):
    x_data["dist_FH_LS"] = joint_distance(x_data, "FH", "LS")
    # Front head        ->  right shoulder      (1->4)
    x_data["dist_FH_RS"] = joint_distance(x_data, "FH", "RS")
    # left shoulder     ->  left wrist          (8->10)
    x_data["dist_LS_LW"] = joint_distance(x_data, "LS", "LW")
    # right shoulder    ->  right wrist         (4->7)
    x_data["dist_RS_RW"] = joint_distance(x_data, "RS", "RW")
    # v sacral          ->  left elbow          (13->9)
    x_data["dist_VS_LE"] = joint_distance(x_data, "VS", "LE")
    # v sacral          ->  right elbow         (13->6)
    x_data["dist_VS_RE"] = joint_distance(x_data, "VS", "RE")
    # v sacral          ->  left wrist          (13->10)
    x_data["dist_VS_LW"] = joint_distance(x_data, "VS", "LW")
    # v sacral          ->  right wrist         (13->7)
    x_data["dist_VS_RW"] = joint_distance(x_data, "VS", "RW")
    # v sacral          ->  rear head           (13->3)
    # x_data["dist_VS_RH"] = joint_distance(x_data, "VS", "RH")
    # v sacral          ->  top head            (13->2)
    # x_data["dist_VS_TH"] = joint_distance(x_data, "VS", "TH")
    # left wrist        ->  right wrist         (10->7)
    x_data["dist_LW_RW"] = joint_distance(x_data, "LW", "RW")
    # left asis         ->  left wrist          (12->10)
    x_data["dist_LA_LW"] = joint_distance(x_data, "LA", "LW")
    # right asis        ->  right wrist         (11->7)
    x_data["dist_RA_RW"] = joint_distance(x_data, "RA", "RW")
    # left wrist        ->  top head            (10->2)
    # x_data["dist_LW_TH"] = joint_distance(x_data, "LW", "TH")
    # # right wrist       ->  top head            (7->2)
    # x_data["dist_RW_TH"] = joint_distance(x_data, "RW", "TH")
    # top head          ->  left asis           (2->12)
    # x_data["dist_TH_LA"] = joint_distance(x_data, "TH", "LA")
    return x_data

In [9]:
def get_all_joint_distances_red_diff(x_data):
    x_data["dist_FH_LS"] = np.nan_to_num(np.append(np.diff(joint_distance(x_data, "FH", "LS")), 0)).tolist()
    x_data["dist_FH_RS"] = np.nan_to_num(np.append(np.diff(joint_distance(x_data, "FH", "RS")), 0)).tolist()
    x_data["dist_LS_LW"] = np.nan_to_num(np.append(np.diff(joint_distance(x_data, "LS", "LW")), 0)).tolist()
    x_data["dist_RS_RW"] = np.nan_to_num(np.append(np.diff(joint_distance(x_data, "RS", "RW")), 0)).tolist()
    x_data["dist_VS_LE"] = np.nan_to_num(np.append(np.diff(joint_distance(x_data, "VS", "LE")), 0)).tolist()
    x_data["dist_VS_RE"] = np.nan_to_num(np.append(np.diff(joint_distance(x_data, "VS", "RE")), 0)).tolist()
    x_data["dist_VS_LW"] = np.nan_to_num(np.append(np.diff(joint_distance(x_data, "VS", "LW")), 0)).tolist()
    x_data["dist_VS_RW"] = np.nan_to_num(np.append(np.diff(joint_distance(x_data, "VS", "RW")), 0)).tolist()
    x_data["dist_LW_RW"] = np.nan_to_num(np.append(np.diff(joint_distance(x_data, "LW", "RW")), 0)).tolist()
    x_data["dist_LA_LW"] = np.nan_to_num(np.append(np.diff(joint_distance(x_data, "LA", "LW")), 0)).tolist()
    x_data["dist_RA_RW"] = np.nan_to_num(np.append(np.diff(joint_distance(x_data, "RA", "RW")), 0)).tolist()
    return x_data

In [10]:
def joint_angle(x_data, joint1, joint2, joint3):
    x1, y1, z1 = x_data[f"{joint1}_X"], x_data[f"{joint1}_Y"], x_data[f"{joint1}_Z"]
    x2, y2, z2 = x_data[f"{joint2}_X"], x_data[f"{joint2}_Y"], x_data[f"{joint2}_Z"]
    x3, y3, z3 = x_data[f"{joint3}_X"], x_data[f"{joint3}_Y"], x_data[f"{joint3}_Z"]
    v1 = np.array([x2-x1, y2-y1, z2-z1]).T
    v2 = np.array([x3-x2, y3-y2, z3-z2]).T
    v1_unit = v1/np.expand_dims(np.linalg.norm(v1, axis=1), axis=1)
    v2_unit = v2/np.expand_dims(np.linalg.norm(v2, axis=1), axis=1)
    angle = np.arccos(np.sum(v1_unit*v2_unit, axis=1)) # dot multiplication
    return angle

def get_all_joint_angles(x_data):
    # joints
    # left shoulder     ->  left elbow      ->  left wrist      (8->9->10)
    x_data["angle_LS_LE_LW"] = joint_angle(x_data, "LS", "LE", "LW")
    # right shoulder    ->  right elbow     ->  right wrist     (4->6->7)
    x_data["angle_RS_RE_RW"] = joint_angle(x_data, "RS", "RE", "RW")
    # right shoulder    ->  left shoulder   ->  front head      (4->8->1)
    x_data["angle_RS_LS_FH"] = joint_angle(x_data, "RS", "LS", "FH")
    # right shoulder    ->  left shoulder   ->  left elbow      (4->8->9)
    x_data["angle_RS_LS_LE"] = joint_angle(x_data, "RS", "LS", "LE")
    # left shoulder     ->  right shoulder  ->  right elbow     (8->4->6)
    x_data["angle_LS_RS_RE"] = joint_angle(x_data, "LS", "RS", "RE")
    # v sacral          ->  right offset    ->  rear head       (13->5->3)
    x_data["angle_VS_RO_RH"] = joint_angle(x_data, "VS", "RO", "RH")
    # vsacral           ->  top head        ->  front head      (13->2->1)
    x_data["angle_VS_TH_FH"] = joint_angle(x_data, "VS", "TH", "FH")
    # v sacral          ->  left shoulder   ->  left elbow      (13->8->9)
    x_data["angle_VS_LS_LE"] = joint_angle(x_data, "VS", "LS", "LE")
    # v sacral          ->  right shoulder  ->  right elbow     (13->4->6)
    x_data["angle_VS_RS_RE"] = joint_angle(x_data, "VS", "RS", "RE")
    # left asis         ->  left shoulder   ->  left elbow      (12->8->9)
    x_data["angle_LA_LS_LE"] = joint_angle(x_data, "LA", "LS", "LE")
    # right asis        -> right shoulder   ->  right elbow     (11->4->6)
    x_data["angle_RA_RS_RE"] = joint_angle(x_data, "RA", "RS", "RE")
    return x_data

In [11]:
def get_all_joint_angles_red(x_data):
    # joints
    x_data["angle_LS_LE_LW"] = joint_angle(x_data, "LS", "LE", "LW")
    x_data["angle_RS_RE_RW"] = joint_angle(x_data, "RS", "RE", "RW")
    x_data["angle_RS_LS_LE"] = joint_angle(x_data, "RS", "LS", "LE")
    x_data["angle_LS_RS_RE"] = joint_angle(x_data, "LS", "RS", "RE")
    x_data["angle_VS_RO_RH"] = joint_angle(x_data, "VS", "RO", "RH")
    return x_data

In [12]:
def get_all_joint_angles_diff_red(x_data):
    # joints
    x_data["ang_dif_LS_LE_LW"] = np.nan_to_num(np.append(np.diff(joint_angle(x_data, "LS", "LE", "LW")), 0)).tolist()
    x_data["ang_dif_RS_RE_RW"] = np.nan_to_num(np.append(np.diff(joint_angle(x_data, "RS", "RE", "RW")), 0)).tolist()
    x_data["ang_dif_RS_LS_LE"] = np.nan_to_num(np.append(np.diff(joint_angle(x_data, "RS", "LS", "LE")), 0)).tolist()
    x_data["ang_dif_LS_RS_RE"] = np.nan_to_num(np.append(np.diff(joint_angle(x_data, "LS", "RS", "RE")), 0)).tolist()
    x_data["ang_dif_VS_RO_RH"] = np.nan_to_num(np.append(np.diff(joint_angle(x_data, "VS", "RO", "RH")), 0)).tolist()
    return x_data

In [20]:
def plane_angles(x_data, joint1, joint2):
    x1, y1, z1 = x_data[f"{joint1}_X"], x_data[f"{joint1}_Y"], x_data[f"{joint1}_Z"]
    x2, y2, z2 = x_data[f"{joint2}_X"], x_data[f"{joint2}_Y"], x_data[f"{joint2}_Z"]
    v = np.array([x2-x1, y2-y1, z2-z1]).T
    vx = np.array([1, 0, 0])
    vy = np.array([0, 1, 0])
    vz = np.array([0, 0, 1])
    v_unit = v/np.expand_dims(np.linalg.norm(v, axis=1), axis=1)
    angle_x = np.arccos(np.sum(v_unit*vx, axis=1)) 
    angle_y = np.arccos(np.sum(v_unit*vy, axis=1)) 
    angle_z = np.arccos(np.sum(v_unit*vz, axis=1)) 
    return angle_x, angle_y, angle_z

def get_all_angles_with_plane(x_data):
    # joints
    # arm angles: left shoulder     ->  left elbow 
    x_data["angle_x_LS_LE"], x_data["angle_y_LS_LE"], x_data["angle_z_LS_LE"] = plane_angles(x_data, "LS", "LE")
    # forearm angles: left elbow    ->  left  wrist
    x_data["angle_x_LE_LW"], x_data["angle_y_LE_LW"], x_data["angle_z_LE_LW"] = plane_angles(x_data, "LE", "LW")
    # arm angles: right shoulder    ->  right elbow 
    x_data["angle_x_RS_RE"], x_data["angle_y_RS_RE"], x_data["angle_z_RS_RE"] = plane_angles(x_data, "RS", "RE")
    # forearm angles: right elbow   ->  right  wrist
    x_data["angle_x_RE_RW"], x_data["angle_y_RE_RW"], x_data["angle_z_RE_RW"] = plane_angles(x_data, "RE", "RW")
    # backbone angles: v sacral     ->  rear head
    # x_data["angle_x_VS_RH"], x_data["angle_y_VS_RH"], x_data["angle_z_VS_RH"] = plane_angles(x_data, "VS", "RH")
    return x_data

In [14]:
def get_all_angles_with_plane_diff(x_data):
    # joints
    # arm angles: left shoulder     ->  left elbow 
    dif = lambda x: np.nan_to_num(np.append(np.diff(x), 0)).tolist()
    a, b, c = plane_angles(x_data, "LS", "LE")
    x_data["angle_x_LS_LE"], x_data["angle_y_LS_LE"], x_data["angle_z_LS_LE"] = dif(a), dif(b), dif(c)
    # forearm angles: left elbow    ->  left  wrist
    a, b, c = plane_angles(x_data, "LE", "LW")
    x_data["angle_x_LE_LW"], x_data["angle_y_LE_LW"], x_data["angle_z_LE_LW"] =  dif(a), dif(b), dif(c)
    # arm angles: right shoulder    ->  right elbow 
    a, b, c = plane_angles(x_data, "RS", "RE")
    x_data["angle_x_RS_RE"], x_data["angle_y_RS_RE"], x_data["angle_z_RS_RE"] =  dif(a), dif(b), dif(c)
    # forearm angles: right elbow   ->  right  wrist
    a, b, c = plane_angles(x_data, "RE", "RW")
    x_data["angle_x_RE_RW"], x_data["angle_y_RE_RW"], x_data["angle_z_RE_RW"] =  dif(a), dif(b), dif(c)
    # backbone angles: v sacral     ->  rear head
    # x_data["angle_x_VS_RH"], x_data["angle_y_VS_RH"], x_data["angle_z_VS_RH"] = plane_angles(x_data, "VS", "RH")
    return x_data

# Pre-processed functions (add/remove streams/features here)

In [14]:
def remove_acc():
    def get_streams(x_data):
        speed, acc = get_speed_acc(x_data)
        x_data = pd.concat([x_data, speed], axis=1)
        x_data = get_all_joint_distances(x_data)
        return x_data

    def get_features(x_data):
        features = []
        cols = x_data.columns.tolist()
        for k in cols:
            features.append(x_data[k].std(ddof=0))
            features.append(np.max(x_data[k]))
            features.append(np.min(x_data[k]))
            features.append(np.median(x_data[k]))        
        return features
    joint_names = ['FH', 'TH', 'RH', 'RS', 'RO', 'RE', 'RW', 'LS', 'LE', 'LW', 'RA', 'LA', 'VS']
    pos_cols = [i+j for i in joint_names for j in ["_X", "_Y", "_Z"]]
    speed_cols = [f"{col}_speed" for col in pos_cols]
    distance_cols = ['dist_FH_LS', 'dist_FH_RS', 'dist_LS_LW', 'dist_RS_RW', 'dist_VS_LE', 'dist_VS_RE', 
        'dist_VS_LW', 'dist_VS_RW','dist_VS_RH', 'dist_VS_TH', 'dist_LW_RW', 'dist_LA_LW', 'dist_RA_RW', 
        'dist_LW_TH', 'dist_RW_TH', 'dist_TH_LA']
    colnames = pos_cols + speed_cols + distance_cols
    return get_streams, get_features, colnames

In [17]:
def angle_dif_red_remove_acc():
    def get_streams(x_data):
        speed, acc = get_speed_acc(x_data)
        x_data = pd.concat([x_data, speed], axis=1)
        x_data = get_all_joint_distances(x_data)
        x_data = get_all_joint_angles_diff_red(x_data)
        return x_data

    def get_features(x_data):
        features = []
        cols = x_data.columns.tolist()
        for k in cols:
            features.append(x_data[k].std(ddof=0))
            features.append(np.max(x_data[k]))
            features.append(np.min(x_data[k]))
            features.append(np.median(x_data[k]))        
        return features
    joint_names = ['FH', 'TH', 'RH', 'RS', 'RO', 'RE', 'RW', 'LS', 'LE', 'LW', 'RA', 'LA', 'VS']
    pos_cols = [i+j for i in joint_names for j in ["_X", "_Y", "_Z"]]
    speed_cols = [f"{col}_speed" for col in pos_cols]
    distance_cols = ['dist_FH_LS', 'dist_FH_RS', 'dist_LS_LW', 'dist_RS_RW', 'dist_VS_LE', 'dist_VS_RE', 
        'dist_VS_LW', 'dist_VS_RW','dist_VS_RH', 'dist_VS_TH', 'dist_LW_RW', 'dist_LA_LW', 'dist_RA_RW', 
        'dist_LW_TH', 'dist_RW_TH', 'dist_TH_LA']
    angle_dif_cols = ["LSLELW", "RSRERW", "RSLSLE", "LSRSRE", "VSRORH"]
    colnames = pos_cols + speed_cols + distance_cols + angle_dif_cols
    return get_streams, get_features, colnames

In [19]:
def dist_red_angle_red_diff_remove_acc():
    def get_streams(x_data):
        speed, acc = get_speed_acc(x_data)
        x_data = pd.concat([x_data, speed], axis=1)
        x_data = get_all_joint_distances_red(x_data)
        x_data = get_all_joint_angles_diff_red(x_data)
        return x_data

    def get_features(x_data):
        features = []
        cols = x_data.columns.tolist()
        for k in cols:
            features.append(x_data[k].std(ddof=0))
            features.append(np.max(x_data[k]))
            features.append(np.min(x_data[k]))
            features.append(np.median(x_data[k]))        
        return features
    joint_names = ['FH', 'TH', 'RH', 'RS', 'RO', 'RE', 'RW', 'LS', 'LE', 'LW', 'RA', 'LA', 'VS']
    pos_cols = [i+j for i in joint_names for j in ["_X", "_Y", "_Z"]]
    speed_cols = [f"{col}_speed" for col in pos_cols]
    distance_cols = ['dist_FH_LS', 'dist_FH_RS', 'dist_LS_LW', 'dist_RS_RW', 'dist_VS_LE', 'dist_VS_RE', 
        'dist_VS_LW', 'dist_VS_RW', 'dist_LW_RW', 'dist_LA_LW', 'dist_RA_RW']
    angle_dif_cols = ["LSLELW", "RSRERW", "RSLSLE", "LSRSRE", "VSRORH"]
    colnames = pos_cols + speed_cols + distance_cols + angle_dif_cols
    return get_streams, get_features, colnames

In [21]:
def dist_red_angle_red_diff_plane_remove_acc():
    def get_streams(x_data):
        speed, acc = get_speed_acc(x_data)
        x_data = pd.concat([x_data, speed], axis=1)
        x_data = get_all_joint_distances_red(x_data)
        x_data = get_all_joint_angles_diff_red(x_data)
        x_data = get_all_angles_with_plane(x_data)
        return x_data

    def get_features(x_data):
        features = []
        cols = x_data.columns.tolist()
        for k in cols:
            features.append(x_data[k].std(ddof=0))
            features.append(np.max(x_data[k]))
            features.append(np.min(x_data[k]))
            features.append(np.median(x_data[k]))        
        return features
    joint_names = ['FH', 'TH', 'RH', 'RS', 'RO', 'RE', 'RW', 'LS', 'LE', 'LW', 'RA', 'LA', 'VS']
    pos_cols = [i+j for i in joint_names for j in ["_X", "_Y", "_Z"]]
    speed_cols = [f"{col}_speed" for col in pos_cols]
    distance_cols = ['dist_FH_LS', 'dist_FH_RS', 'dist_LS_LW', 'dist_RS_RW', 'dist_VS_LE', 'dist_VS_RE', 
        'dist_VS_LW', 'dist_VS_RW', 'dist_LW_RW', 'dist_LA_LW', 'dist_RA_RW']
    angle_dif_cols = ["LSLELW", "RSRERW", "RSLSLE", "LSRSRE", "VSRORH"]
    plane_cols = ["xLSLE", "yLSLE", "zLSLE", "xLELW", "yLELW", "zLELW", "xRSRE", "yRSRE", "zRSRE", "xRERW", "yRERW", "zRERW"]
    colnames = pos_cols + speed_cols + distance_cols + angle_dif_cols + plane_cols
    return get_streams, get_features, colnames

In [22]:
def dist_red_dist_red_diff_angle_red_diff_remove_acc():
    def get_streams(x_data):
        speed, acc = get_speed_acc(x_data)
        x_data = pd.concat([x_data, speed], axis=1)
        x_data = get_all_joint_distances_red(x_data)
        x_data = get_all_joint_distances_red_diff(x_data)
        x_data = get_all_joint_angles_diff_red(x_data)
        return x_data

    def get_features(x_data):
        features = []
        cols = x_data.columns.tolist()
        for k in cols:
            features.append(x_data[k].std(ddof=0))
            features.append(np.max(x_data[k]))
            features.append(np.min(x_data[k]))
            features.append(np.median(x_data[k]))        
        return features
    joint_names = ['FH', 'TH', 'RH', 'RS', 'RO', 'RE', 'RW', 'LS', 'LE', 'LW', 'RA', 'LA', 'VS']
    pos_cols = [i+j for i in joint_names for j in ["_X", "_Y", "_Z"]]
    speed_cols = [f"{col}_speed" for col in pos_cols]
    distance_cols = ['dist_FH_LS', 'dist_FH_RS', 'dist_LS_LW', 'dist_RS_RW', 'dist_VS_LE', 'dist_VS_RE', 
        'dist_VS_LW', 'dist_VS_RW', 'dist_LW_RW', 'dist_LA_LW', 'dist_RA_RW']
    distance_diff_cols = ["diff_"+i for i in distance_cols]
    angle_dif_cols = ["LSLELW", "RSRERW", "RSLSLE", "LSRSRE", "VSRORH"]
    colnames = pos_cols + speed_cols + distance_cols + distance_diff_cols + angle_dif_cols
    return get_streams, get_features, colnames

# Random forest

## <p style="color:red"> <b> bayes optimization: dist reduced + angle difference reduced + remove acceleration </b> </p>

In [103]:
get_streams, get_features, colnames = dist_red_angle_red_diff_remove_acc()
overlap_rate, window_size = 0.8, 4000
X, y, file_lengths = get_processed_dataset(overlap_rate, window_size)

loading the data...	

100%|██████████| 151/151 [00:13<00:00, 11.13it/s]


extracting the features...  

100%|██████████| 512/512 [00:30<00:00, 16.66it/s]


In [127]:
def eval_function(n_estimators, criterion, max_depth, min_samples_split, min_samples_leaf):
    model = RFC(
                n_estimators=int(max(n_estimators,0)), 
                criterion="entropy" if criterion>=0.5 else "gini",                                                             
                max_depth=int(max(max_depth, 1)),
                min_samples_split=int(max(min_samples_split,2)), 
                min_samples_leaf=int(min_samples_leaf),
                n_jobs=-1)
    scores = scaled_model_evaluator(model, X, y, file_lengths, n_repeats=5, voting=True, verbose=True)
    return np.median(scores)

# need to give the range in the tuples
parameters = {"n_estimators": (1000, 5000),
                "max_depth": (8, 60),
                "min_samples_split": (2, 10),
                "criterion": (0, 1),
                "min_samples_leaf": (1,8)
                }


In [ ]:
best_solution = bayesian_optimization(eval_function, parameters) 

In [130]:
best_solution

{'target': 0.8097345132743363,
 'params': {'criterion': 0.18041252076428138,
  'max_depth': 48.10632563606063,
  'min_samples_leaf': 5.693290179489262,
  'min_samples_split': 2.21796460693196,
  'n_estimators': 2332.4339379229414}}

In [131]:
model = RFC(2332, criterion="gini", max_depth=48, min_samples_leaf=5, min_samples_split=2, n_jobs=-1)
scores = scaled_model_evaluator(model, X, y, file_lengths, n_repeats=10, voting=True, verbose=True)

100%|██████████| 10/10 [02:49<00:00, 16.94s/it]

Mean Score: 0.8005794824269304
Std Score: 0.015106236114833492
Min Score: 0.7669172932330827
Max Score: 0.827433628318584
